### ECG & PCG analysis

Library imports


In [41]:
from sklearn.metrics import roc_curve
import numpy as np
import os
from keras.models import Sequential, Model
import pandas as pd
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix as cm, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import itertools
from sklearn.model_selection import GroupShuffleSplit
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.utils import class_weight
from keras.optimizers import Adam, SGD
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedGroupKFold, GridSearchCV
from itertools import product
from keras.applications import VGG16
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
import pickle
from sklearn.utils import compute_class_weight
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import EarlyStopping




In [18]:
main_folder = 'pcg_circor'

### PCG Model



In [19]:
train_df_p = pd.read_csv(f"./{main_folder}/train/dataset.csv",usecols=range(1,3))
test_df_p = pd.read_csv(f"./{main_folder}/test/dataset.csv",usecols=range(1,3))


In [20]:
train_df_p['label']=train_df_p['label'].astype(str)
test_df_p['label']=test_df_p['label'].astype(str)


In [21]:
train_df_p['group'] = train_df_p['filename'].apply(lambda x: x.split('_')[0])
test_df_p['group'] = test_df_p['filename'].apply(lambda x: x.split('_')[0])

In [22]:
train_df_p

,filename,label,group
0,3_8.tiff,1,3
1,5_14.tiff,1,5
2,2_7.tiff,1,2
3,1_3.tiff,1,1
4,5_13.tiff,1,5
...,...,...,...
8970,2199_8970.tiff,1,2199
8971,2200_8972.tiff,1,2200
8972,2200_8973.tiff,1,2200
8973,2199_8971.tiff,1,2199


In [24]:
train_datagen_p = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen_p = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen_p = ImageDataGenerator(preprocessing_function=preprocess_input)


In [25]:
train_df_p['label'].value_counts()

0    4878
1    4097
Name: label, dtype: int64

In [26]:
# Define your model creation function
def create_model(optimizer='adam', learning_rate=0.001, dropout = 0.5, neurons = 128):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    
    x = base_model.output
    x = Flatten()(x)  # Add Global Average Pooling
    x = Dense(neurons, activation='relu')(x)  # Add a fully connected layer
    x = Dropout(dropout) (x)
    predictions = Dense(1, activation='sigmoid')(x)  # Replace softmax with sigmoid for binary classification

    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
        
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['Accuracy'])
    return model


In [27]:
# Define your model creation function
def create_model_finetune(optimizer='adam', learning_rate=0.001, dropout = 0.5, neurons = 128):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
        
    base_model.trainable = True
    num_layers = len(base_model.layers)
    #num_layers_fine_tune = 8
    num_layers_fine_tune = 18
    for model_layer in base_model.layers[:num_layers - num_layers_fine_tune]:
        model_layer.trainable = False
    
    
        
        
    x = base_model.output
    
    
    x = Flatten()(x)  # Add Global Average Pooling
    x = Dense(neurons, activation='relu')(x)  # Add a fully connected layer
    x = Dropout(dropout) (x)
    predictions = Dense(1, activation='sigmoid')(x)  # Replace softmax with sigmoid for binary classification

    model = Model(inputs=base_model.input, outputs=predictions)
    
        
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['Accuracy'])
    return model


In [28]:
model = create_model_finetune()

In [29]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [30]:
class ScoreCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, train_data):
        super().__init__()
        self.validation_data = validation_data
        self.train_data = train_data
        self.val_f1_scores = []
        self.train_f1_scores = []
        self.val_accuracies = []
        self.train_accuracies = []
        
    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data, self.validation_data.classes
        X_train, y_train = self.train_data, self.train_data.classes

        y_pred_val = self.model.predict(X_val)
        y_pred_rounded_val = np.round(y_pred_val)  # Round predictions to binary values
        y_pred_train = self.model.predict(X_train)
        y_pred_rounded_train = np.round(y_pred_train)  # Round predictions to binary values
        
        macro_f1_val = f1_score(y_val, y_pred_rounded_val, average='macro')
        self.val_f1_scores.append(macro_f1_val)
        
        macro_f1_train = f1_score(y_train, y_pred_rounded_train, average='macro')
        self.train_f1_scores.append(macro_f1_train)
        
        acc_val = accuracy_score(y_val, y_pred_rounded_val)
        self.val_accuracies.append(acc_val)
        acc_train = accuracy_score(y_train, y_pred_rounded_train)
        self.train_accuracies.append(acc_train)

In [42]:
# Define the parameter grid
param_grid = {
    'batch_size': [ 32],
    'epochs': [100],
    'optimizer': ['adam'],
    'learning_rate': [  0.000001],
    'dropout':[ 0.5],
    'neurons':[32],
    'class_weights':[False]
}

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# Store results
results = []



sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True )


In [39]:
def calculate_scores(generator, model):
    y_true = generator.classes
    predictions = model.predict(generator)
    threshold = 0.5
    predicted_classes = (predictions > threshold).astype(int)
    
    f1_value = f1_score(y_true,predicted_classes, average='binary' )
    acc = accuracy_score(y_true, predicted_classes)
    auc = roc_auc_score(y_true, predictions) 
    precision = precision_score(y_true, predicted_classes)
    recall = recall_score(y_true, predicted_classes)
    return f1_value, acc, auc, precision, recall, predictions, y_true
    

In [43]:

for params in product(*param_grid.values()):
    detailed_results = []
    param_dict = dict(zip(param_grid.keys(), params))
    
    batch_size = param_dict['batch_size']
    n_epochs = param_dict['epochs']
    use_class_weights = param_dict['class_weights']
    param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}
    
    
    epoch_info = []
    
    #For each fold
    for fold_idx, (train_indices, test_indices) in enumerate(sgkf.split(train_df_p['filename'], train_df_p['label'], groups=train_df_p['group'])):
        
        print(param_dict)
        print (f'-----------------FOLD {fold_idx}-----------------')
        
        model = create_model_finetune(**param_dict_filtered)


        train_gen = train_datagen_p.flow_from_dataframe(dataframe=train_df_p.iloc[train_indices],
                                             directory=f"./{main_folder}/train",
                                            target_size=(224, 224),
                                             x_col = 'filename',
                                             y_col = 'label',
                                             class_mode = 'binary',
                                             classes = ["0","1"], shuffle=False, batch_size=batch_size,
                                             color_mode = 'rgb' )



        test_gen = train_datagen_p.flow_from_dataframe(dataframe=train_df_p.iloc[test_indices],
                                             directory=f"./{main_folder}/train",
                                            target_size=(224, 224),
                                             x_col = 'filename',
                                             y_col = 'label',
                                             class_mode = 'binary',
                                             shuffle = False,
                                             classes = ["0","1"], batch_size=batch_size,
                                             color_mode = 'rgb')
        
        train_group = train_df_p.iloc[train_indices]['group']
        val_group = train_df_p.iloc[test_indices]['group']
        
        if use_class_weights:
            y_train = train_gen.classes

            class_weights = compute_class_weight('balanced',
                                                     classes=np.unique(y_train),
                                                     y=y_train)
            
            
            class_weights = {i:w for i,w in enumerate(class_weights)}
            #score_callback = ScoreCallback(test_gen, train_gen)

            #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
            history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen,class_weight=class_weights, callback=[early_stopping])
        else:
            history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,validation_data=test_gen, callbacks=[early_stopping])
        
        
        train_gen.reset()
        test_gen.reset()
        train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(train_gen, model)
        
        test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(test_gen, model)
        
        #test_predictions = model.predict(test_gen)
        
        
        
        print ( train_f1, train_acc, train_auc, train_precision,train_recall)
        
        print (test_f1, test_acc, test_auc, test_precision, test_recall)
        
        
        
        epoch_losses = history.history['loss']
        
        epoch_accuracy = history.history['Accuracy']
        epoch_val_accuracy = history.history['val_Accuracy']
        #epoch_accuracies = score_callback.accuracies_train
        #epoch_f1_macro = score_callback.train_f1_scores
        epoch_val_losses = history.history['val_loss']
        #epoch_val_accuracies = score_callback.accuracies_val
        #epoch_val_f1_macro = score_callback.val_f1_scores
        # After training
        #print("F1-scores per epoch:", score_callback.val_f1_scores)
        #print("Accuracies per epoch:", accuracy_callback.accuracies)
        
        epoch_info.append({
            'params': param_dict,
            'fold_number': fold_idx + 1,
            'loss': epoch_losses,
            #'val_loss':epoch_val_losses,
            'acc_train': epoch_accuracy,
            #'f1_train': epoch_f1_macro,
            'acc_val': epoch_val_accuracy,
            #'f1_val': epoch_val_f1_macro,
            'epoch_val_losses':epoch_val_losses,
            'train_f1': train_f1,
            'train_acc': train_acc,
            'test_f1': test_f1,
            'test_acc': test_acc,
            'train_auc':train_auc,
            'test_auc':test_auc,
            'train_precision':train_precision,
            'train_recall':train_recall,
            'test_precision':test_precision,
            'test_recall':test_recall,
            'y_true_train':y_true_train,
            'y_true_val':y_true_val, #y_true_val
            'prediction_train': prediction_train,
            'prediction_test': prediction_test,
            'train_group':train_group,
            'test_group':val_group
            
        })
    
    detailed_results.extend(epoch_info)

    # Convert detailed_results to a DataFrame
    detailed_results_df = pd.DataFrame(detailed_results)

    print(detailed_results_df)
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
    print("date and time =", dt_string)


    detailed_results_df.to_csv(main_folder+'_vgg.csv')
    with open (main_folder+dt_string+'cvcircor_vgg.pkl', 'wb') as f:
        pickle.dump(detailed_results_df, f)


{'batch_size': 32, 'epochs': 100, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': False}
-----------------FOLD 0-----------------
Found 7179 validated image filenames belonging to 2 classes.
Found 1796 validated image filenames belonging to 2 classes.
Epoch 1/100
225/225 [==============================] - 114s 501ms/step - loss: 2.0451 - Accuracy: 0.5226 - val_loss: 0.8110 - val_Accuracy: 0.4978
Epoch 2/100
225/225 [==============================] - 117s 522ms/step - loss: 0.7930 - Accuracy: 0.5192 - val_loss: 0.7194 - val_Accuracy: 0.5089
Epoch 3/100
225/225 [==============================] - 117s 518ms/step - loss: 0.7198 - Accuracy: 0.5306 - val_loss: 0.6990 - val_Accuracy: 0.5379
Epoch 4/100
225/225 [==============================] - 116s 515ms/step - loss: 0.6993 - Accuracy: 0.5424 - val_loss: 0.6906 - val_Accuracy: 0.5551
Epoch 5/100
225/225 [==============================] - 116s 517ms/step - loss: 0.6936 - Accuracy: 0.5435 - val_loss

225/225 [==============================] - 112s 498ms/step - loss: 0.5906 - Accuracy: 0.6445 - val_loss: 0.6882 - val_Accuracy: 0.5895
Epoch 26/100
225/225 [==============================] - 112s 497ms/step - loss: 0.5885 - Accuracy: 0.6441 - val_loss: 0.6832 - val_Accuracy: 0.6006
Epoch 27/100
225/225 [==============================] - 112s 498ms/step - loss: 0.5881 - Accuracy: 0.6476 - val_loss: 0.6889 - val_Accuracy: 0.5967
Epoch 28/100
225/225 [==============================] - 112s 497ms/step - loss: 0.5780 - Accuracy: 0.6551 - val_loss: 0.6777 - val_Accuracy: 0.5956
Epoch 29/100
225/225 [==============================] - 112s 497ms/step - loss: 0.5730 - Accuracy: 0.6551 - val_loss: 0.6824 - val_Accuracy: 0.5889
Epoch 30/100
225/225 [==============================] - 112s 498ms/step - loss: 0.5648 - Accuracy: 0.6594 - val_loss: 0.6928 - val_Accuracy: 0.5900
Epoch 31/100
225/225 [==============================] - 112s 498ms/step - loss: 0.5687 - Accuracy: 0.6651 - val_loss: 0.6838 

225/225 [==============================] - 115s 510ms/step - loss: 0.8292 - Accuracy: 0.5250 - val_loss: 0.6945 - val_Accuracy: 0.5660
Epoch 3/100
225/225 [==============================] - 115s 509ms/step - loss: 0.7214 - Accuracy: 0.5422 - val_loss: 0.6811 - val_Accuracy: 0.5794
Epoch 4/100
225/225 [==============================] - 115s 510ms/step - loss: 0.6992 - Accuracy: 0.5603 - val_loss: 0.6782 - val_Accuracy: 0.5822
Epoch 5/100
225/225 [==============================] - 115s 510ms/step - loss: 0.6877 - Accuracy: 0.5620 - val_loss: 0.6743 - val_Accuracy: 0.5962
Epoch 6/100
225/225 [==============================] - 115s 509ms/step - loss: 0.6811 - Accuracy: 0.5684 - val_loss: 0.6726 - val_Accuracy: 0.5923
Epoch 7/100
225/225 [==============================] - 115s 511ms/step - loss: 0.6764 - Accuracy: 0.5794 - val_loss: 0.6700 - val_Accuracy: 0.5984
Epoch 8/100
225/225 [==============================] - 115s 510ms/step - loss: 0.6762 - Accuracy: 0.5760 - val_loss: 0.6692 - val_

224/224 [==============================] - 114s 510ms/step - loss: 0.6522 - Accuracy: 0.6010 - val_loss: 0.6667 - val_Accuracy: 0.5977
Epoch 15/100
224/224 [==============================] - 123s 549ms/step - loss: 0.6502 - Accuracy: 0.6003 - val_loss: 0.6676 - val_Accuracy: 0.5901
Epoch 16/100
224/224 [==============================] - 119s 531ms/step - loss: 0.6414 - Accuracy: 0.6126 - val_loss: 0.6635 - val_Accuracy: 0.6032
Epoch 17/100
224/224 [==============================] - 122s 543ms/step - loss: 0.6405 - Accuracy: 0.6185 - val_loss: 0.6648 - val_Accuracy: 0.6010
Epoch 18/100
224/224 [==============================] - 116s 518ms/step - loss: 0.6361 - Accuracy: 0.6209 - val_loss: 0.6657 - val_Accuracy: 0.5961
Epoch 19/100
224/224 [==============================] - 116s 519ms/step - loss: 0.6260 - Accuracy: 0.6328 - val_loss: 0.6633 - val_Accuracy: 0.5983
Epoch 20/100
224/224 [==============================] - 119s 533ms/step - loss: 0.6260 - Accuracy: 0.6273 - val_loss: 0.6601 

In [ ]:
with open (r'.\physionet2016_bcdef01_09_2023__13_23_13_vgg.pkl', 'rb') as f:
    df = pickle.load(f)

### Retrain Selected model on full training data

In [44]:

detailed_results=[]
param_dict = {'batch_size': 32, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': False}
param_str = str(param_dict)
batch_size = param_dict['batch_size']
n_epochs = param_dict['epochs']
use_class_weights = param_dict['class_weights']
param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}
    
    
test_info = []     
model = create_model_finetune(**param_dict_filtered)
epoch_info = []

train_gen = train_datagen_p.flow_from_dataframe(dataframe=train_df_p,
                                     directory=f"./{main_folder}/train",
                                    target_size=(224, 224),
                                     x_col = 'filename',
                                     y_col = 'label',
                                     class_mode = 'binary',
                                     classes = ["0","1"], shuffle=False, batch_size=batch_size,
                                     color_mode = 'rgb' )



test_gen = train_datagen_p.flow_from_dataframe(dataframe=test_df_p,
                                 directory=f"./{main_folder}/test",
                                target_size=(224, 224),
                                 x_col = 'filename',
                                 y_col = 'label',
                                 class_mode = 'binary',
                                 shuffle = False,
                                 classes = ["0","1"], batch_size=batch_size,
                                 color_mode = 'rgb')

#train_group = train_df_p['group']
#score_callback = ScoreCallback(test_gen, train_gen)



if use_class_weights:
    y_train = train_gen.classes

    class_weights = compute_class_weight('balanced',
                                             classes=np.unique(y_train),
                                             y=y_train)


    class_weights = {i:w for i,w in enumerate(class_weights)}
    #score_callback = ScoreCallback(test_gen, train_gen)

    #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
    history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs,class_weight=class_weights, validation_data= test_gen )
else:
    history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, validation_data= test_gen)

train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(train_gen, model)

test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(test_gen, model)


epoch_info.append({
            'params': param_dict,
            'fold_number': -1,
            'loss': epoch_losses,
            #'val_loss':epoch_val_losses,
            'acc_train': epoch_accuracy,
            #'f1_train': epoch_f1_macro,
            'acc_val': epoch_val_accuracy,
            #'f1_val': epoch_val_f1_macro,
            'epoch_val_losses':epoch_val_losses,
            'train_f1': train_f1,
            'train_acc': train_acc,
            'test_f1': test_f1,
            'test_acc': test_acc,
            'train_auc':train_auc,
            'test_auc':test_auc,
            'train_precision':train_precision,
            'train_recall':train_recall,
            'test_precision':test_precision,
            'test_recall':test_recall,
            'y_true_train':y_true_train,
            'y_true_val':y_true_val, #y_true_val
            'prediction_train': prediction_train,
            'prediction_test': prediction_test,
            'train_group':train_group,
            'test_group':val_group
            
        })


detailed_results.extend(epoch_info)

# Convert detailed_results to a DataFrame
detailed_results_df = pd.DataFrame(detailed_results)

print(detailed_results_df)
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
print("date and time =", dt_string)


detailed_results_df.to_csv(main_folder+'_vgg.csv')
with open (main_folder+dt_string+'full_train_vgg.pkl', 'wb') as f:
    pickle.dump(detailed_results_df, f)

#test_predictions = model.predict(test_gen)


#print ( train_f1, train_acc, train_auc, train_precision,train_recall)

#print (test_f1, test_acc, test_auc, test_precision, test_recall)


#model.save(f'pcg_only_finetune.h5')  # creates a HDF5 file 'my_model.h5'



Found 8975 validated image filenames belonging to 2 classes.
Found 3970 validated image filenames belonging to 2 classes.
Epoch 1/20
281/281 [==============================] - 173s 612ms/step - loss: 1.7370 - Accuracy: 0.5180 - val_loss: 0.7870 - val_Accuracy: 0.5025
Epoch 2/20
281/281 [==============================] - 155s 550ms/step - loss: 0.7967 - Accuracy: 0.5199 - val_loss: 0.7288 - val_Accuracy: 0.4834
Epoch 3/20
281/281 [==============================] - 176s 628ms/step - loss: 0.7214 - Accuracy: 0.5460 - val_loss: 0.7083 - val_Accuracy: 0.4806
Epoch 4/20
281/281 [==============================] - 173s 615ms/step - loss: 0.6970 - Accuracy: 0.5625 - val_loss: 0.7074 - val_Accuracy: 0.4710
Epoch 5/20
281/281 [==============================] - 169s 603ms/step - loss: 0.6892 - Accuracy: 0.5611 - val_loss: 0.7028 - val_Accuracy: 0.4791
Epoch 6/20
281/281 [==============================] - 151s 537ms/step - loss: 0.6827 - Accuracy: 0.5662 - val_loss: 0.7021 - val_Accuracy: 0.4834
Ep

In [45]:
detailed_results=[]
param_dict ={'batch_size': 32, 'epochs': 20, 'optimizer': 'adam', 'learning_rate': 1e-06, 'dropout': 0.5, 'neurons': 32, 'class_weights': False}
param_str = str(param_dict)
batch_size = param_dict['batch_size']
n_epochs = param_dict['epochs']
use_class_weights = param_dict['class_weights']
param_dict_filtered = {x:param_dict[x] for x in param_dict.keys() if x not in ('batch_size', 'epochs', 'class_weights')}
    
    
test_info = []     
model = create_model_finetune(**param_dict_filtered)


train_gen = train_datagen_p.flow_from_dataframe(dataframe=train_df_p,
                                     directory=f"./{main_folder}/train",
                                    target_size=(224, 224),
                                     x_col = 'filename',
                                     y_col = 'label',
                                     class_mode = 'binary',
                                     classes = ["0","1"], shuffle=False, batch_size=batch_size,
                                     color_mode = 'rgb' )



test_gen = train_datagen_p.flow_from_dataframe(dataframe=test_df_p,
                                 directory=f"./{main_folder}/test",
                                target_size=(224, 224),
                                 x_col = 'filename',
                                 y_col = 'label',
                                 class_mode = 'binary',
                                 shuffle = False,
                                 classes = ["0","1"], batch_size=batch_size,
                                 color_mode = 'rgb')

#train_group = train_df_p['group']
#score_callback = ScoreCallback(test_gen, train_gen)


class CombinedGen():
    def __init__(self, *gens):
        self.gens = gens

    def generate(self):
        while True:
            for g in self.gens:
                yield next(g)

    def __len__(self):
        return sum([len(g) for g in self.gens])

full_data_generator=CombinedGen(train_gen, test_gen)


if use_class_weights:
    y_train = train_gen.classes

    class_weights = compute_class_weight('balanced',
                                             classes=np.unique(y_train),
                                             y=y_train)


    class_weights = {i:w for i,w in enumerate(class_weights)}
    #score_callback = ScoreCallback(test_gen, train_gen)

    #history = model.fit(train_gen, batch_size = batch_size, epochs=n_epochs, callbacks=[score_callback])
    history = model.fit(full_data_generator.generate(), batch_size = batch_size, epochs=n_epochs,class_weight=class_weights, steps_per_epoch=len(train_gen)+len(test_gen))
else:
    history = model.fit(full_data_generator.generate(), batch_size = batch_size, epochs=n_epochs, steps_per_epoch=len(train_gen)+len(test_gen))

#train_f1, train_acc, train_auc, train_precision, train_recall, prediction_train, y_true_train = calculate_scores(train_gen, model)

#test_f1, test_acc, test_auc, test_precision, test_recall, prediction_test, y_true_val = calculate_scores(test_gen, model)

#test_predictions = model.predict(test_gen)


#print ( train_f1, train_acc, train_auc, train_precision,train_recall)

#print (test_f1, test_acc, test_auc, test_precision, test_recall)


model.save(f'pcg_only_finetune_circor.h5')  # creates a HDF5 file 'my_model.h5'



Found 8975 validated image filenames belonging to 2 classes.
Found 3970 validated image filenames belonging to 2 classes.
Epoch 1/20
406/406 [==============================] - 210s 508ms/step - loss: 1.5338 - Accuracy: 0.5152
Epoch 2/20
406/406 [==============================] - 211s 520ms/step - loss: 0.7253 - Accuracy: 0.5128
Epoch 3/20
406/406 [==============================] - 213s 525ms/step - loss: 0.7027 - Accuracy: 0.5048
Epoch 4/20
406/406 [==============================] - 218s 537ms/step - loss: 0.6909 - Accuracy: 0.5104
Epoch 5/20
406/406 [==============================] - 190s 468ms/step - loss: 0.6890 - Accuracy: 0.5101
Epoch 6/20
406/406 [==============================] - 195s 481ms/step - loss: 0.6846 - Accuracy: 0.5228
Epoch 7/20
406/406 [==============================] - 189s 466ms/step - loss: 0.6783 - Accuracy: 0.5210
Epoch 8/20
406/406 [==============================] - 185s 456ms/step - loss: 0.6756 - Accuracy: 0.5400
Epoch 9/20
406/406 [==========================

In [ ]:


from datetime import datetime
import pickle

now = datetime.now()

# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")
print("date and time =", dt_string)


detailed_results_df.to_csv(main_folder+'_vgg.csv')





In [ ]:
with open (main_folder+dt_string+'_vgg.pkl', 'rb') as f:
    df =pickle.load(f)

In [27]:
detailed_results_df.to_csv('resultados_pcg_2016.csv')



In [30]:
detailed_results_df['test_f1'].mean()

0.7752502345262925

In [ ]:
for i in range (1, 100):
    